# Personalize Workshop Cleanup

This notebook will walk through deleting all of the resources created by the [Personalize workshop](./1.1-Personalize.ipynb). You should only need to perform these steps if you have deployed the Retail Demo Store in your own AWS account and want to deprovision the Personalize resources. If you are participating in an AWS-led workshop, this process is likely not necessary.

Resources have to deleted in a specific sequence to avoid dependency errors. In order, we will delete campaigns, solutions, event trackers, datasets, and the dataset group. In addition, we need to make sure that each resource type is fully deleted before moving on to the next resource type. We'll also delete the schemas for our datasets and the IAM roles we created for Personalize.

## Import Dependencies

To get started, let's import the dependencies we'll need for this notebook.

In [ ]:
# Import Dependencies
import botocore
import boto3
import time
import sys

from packaging import version
from botocore.exceptions import ClientError

# Setup Clients
personalize = boto3.client('personalize')
ssm = boto3.client('ssm')

Since we require a newer version of the botocore library, upgrade the local version if necessary. Note that if the botocore is upgraded, you will need to restart the Jupyter notebook kernel and re-execute the cells from the top to resume. An assertion error is thrown as a reminder if a kernel restart is required.

In [ ]:
# Minimum version of botocore we need for this workshop.
min_botocore_version = '1.16.24'

if version.parse(botocore.__version__) < version.parse(min_botocore_version):
    print('Current version of botocore ({}) does not meet the minimum required version ({})'.format(botocore.__version__, min_botocore_version))
    print('Upgrading to latest pip and botocore...')

    !{sys.executable} -m pip install --upgrade pip
    !{sys.executable} -m pip install --upgrade --no-deps --force-reinstall botocore
    
    assert False, 'Restart the notebook kernel to pick up the latest botocore and begin at the top of this notebook'
else:
    print('Version of botocore ({}) meets minimum requirement ({}) for this notebook'.format(botocore.__version__, min_botocore_version))

## Lookup Dataset Group ARN

Since we only want to delete resources created within the dataset group, let's lookup the dataset group ARN for our dataset group.

In [ ]:
# Name of dataset group that was created in the Personalize workshop notebook.
dataset_group_name = 'retaildemostore'
dataset_group_arn = None

dataset_groups_paginator = personalize.get_paginator('list_dataset_groups')
for dataset_groups_page in dataset_groups_paginator.paginate():
    for dataset_group in dataset_groups_page['datasetGroups']:
        if dataset_group['name'] == dataset_group_name:
            dataset_group_arn = dataset_group['datasetGroupArn']
            break
            
assert dataset_group_arn is not None, 'Could not find dataset group; does it exist?'

print('Dataset Group Arn: ' + dataset_group_arn)

## Delete Filters

Start by deleting all filters associated with the dataset group.

In [ ]:
filters_response = personalize.list_filters(datasetGroupArn = dataset_group_arn, maxResults = 100)
for filter in filters_response['Filters']:
    print('Deleting filter: ' + filter['filterArn'])
    personalize.delete_filter(filterArn = filter['filterArn'])
    
print('Filters deleted')

Clear the SSM parameter so the Recommendations service will no longer try to use it.

In [ ]:
print('Clearing purchased product filter arn SSM parameter')
ssm.put_parameter(
    Name='retaildemostore-personalize-filter-purchased-arn',
    Description='Retail Demo Store Personalize Filter Purchased Products Arn Parameter',
    Value='NONE',
    Type='String',
    Overwrite=True
)

## Determine Solutions

Next let's gather the solution ARNs for all solutions in our dataset group.

In [ ]:
solution_arns = []

solutions_response = personalize.list_solutions(datasetGroupArn = dataset_group_arn, maxResults = 100)
if 'solutions' in solutions_response:
    for solution in solutions_response['solutions']:
        solution_arns.append(solution['solutionArn'])
        
print('Solutions found: ' + str(solution_arns))

## Delete Campaigns for Solutions

For each solution, let's delete all associated campaigns.

In [ ]:
campaign_arns_deleted = []

for solution_arn in solution_arns:
    campaigns_response = personalize.list_campaigns(solutionArn = solution_arn, maxResults = 100)
    
    if 'campaigns' in campaigns_response:
        for campaign in campaigns_response['campaigns']:
            print('Deleting campaign: ' + campaign['campaignArn'])
            
            personalize.delete_campaign(campaignArn = campaign['campaignArn'])
            campaign_arns_deleted.append(campaign['campaignArn'])

if len(campaign_arns_deleted) > 0:
    print('Done initiating deletes for {} campaign(s)'.format(len(campaign_arns_deleted)))
else:
    print('No campaigns to delete')

### Clear SSM Parameters

Since the Retail Demo Store Recommendations service uses SSM parameters to determine the campaigns to use for product recommendations and search personalization, let's clear those parameters so we don't break the UI.

In [ ]:
print('Clearing related products campaign arn SSM parameter')
ssm.put_parameter(
    Name='retaildemostore-related-products-campaign-arn',
    Description='Retail Demo Store Related Products Campaign Arn Parameter',
    Value='NONE',
    Type='String',
    Overwrite=True
)
print('Clearing product recommendation campaign arn SSM parameter')
ssm.put_parameter(
    Name='retaildemostore-product-recommendation-campaign-arn',
    Description='Retail Demo Store Product Recommendation Campaign Arn Parameter',
    Value='NONE',
    Type='String',
    Overwrite=True
)
print('Clearing personalized ranking campaign arn SSM parameter')
response = ssm.put_parameter(
    Name='retaildemostore-personalized-ranking-campaign-arn',
    Description='Retail Demo Store Personalized Ranking Campaign Arn Parameter',
    Value='NONE',
    Type='String',
    Overwrite=True
)

### Wait for Campaigns to be Deleted

In [ ]:
print('Waiting for {} campaigns to be fully deleted'.format(len(campaign_arns_deleted)))
while len(campaign_arns_deleted) > 0:
    try:
        status = None
        max_time = time.time() + 3*60*60 # 3 hours
        while time.time() < max_time:
            describe_campaign_response = personalize.describe_campaign(
                campaignArn = campaign_arns_deleted[0]
            )
            status = describe_campaign_response["campaign"]["status"]
            print('Campaign {}: {}'.format(campaign_arns_deleted[0], status))
            time.sleep(10)
            
        print('Exceeded wait time for campaign {} to delete; skipping'.format(campaign_arns_deleted[0]))
        campaign_arns_deleted.pop(0)

    except ClientError as e:
        error_code = e.response['Error']['Code']
        if error_code == 'ResourceNotFoundException':
            print('Campaign {} no longer exists'.format(campaign_arns_deleted[0]))
            campaign_arns_deleted.pop(0)
            
print('Done deleting campaigns')

## Delete Solutions

With the campaigns fully deleted, we can now delete the solutions for our dataset group.

In [ ]:
solution_arns_deleted = []

for solution_arn in solution_arns:
    print('Deleting solution: ' + solution_arn)

    personalize.delete_solution(solutionArn = solution_arn)
    solution_arns_deleted.append(solution_arn)

if len(solution_arns_deleted) > 0:
    print('Done initiating deletes for {} solution(s)'.format(len(solution_arns_deleted)))
else:
    print('No solutions to delete')

### Wait for Solutions to be Deleted

Before we can move on, ensure the solutions have been fully deleted.

In [ ]:
print('Waiting for {} solutions to be fully deleted'.format(len(solution_arns_deleted)))
while len(solution_arns_deleted) > 0:
    try:
        status = None
        max_time = time.time() + 3*60*60 # 3 hours
        while time.time() < max_time:
            describe_solution_response = personalize.describe_solution(
                solutionArn = solution_arns_deleted[0]
            )
            status = describe_solution_response["solution"]["status"]
            print('Solution {}: {}'.format(solution_arns_deleted[0], status))
            time.sleep(10)
            
        print('Exceeded wait time for solution {} to delete; skipping'.format(solution_arns_deleted[0]))
        solution_arns_deleted.pop(0)

    except ClientError as e:
        error_code = e.response['Error']['Code']
        if error_code == 'ResourceNotFoundException':
            print('Solution {} no longer exists'.format(solution_arns_deleted[0]))
            solution_arns_deleted.pop(0)
            
print('Done deleting solutions')

## Delete Event Trackers

Next we'll delete any event trackers.

In [ ]:
print('Deleting event trackers for dataset group')
event_trackers = 0
event_trackers_paginator = personalize.get_paginator('list_event_trackers')
for event_tracker_page in event_trackers_paginator.paginate(datasetGroupArn = dataset_group_arn):
    for event_tracker in event_tracker_page['eventTrackers']:
        print('Deleting event tracker {}'.format(event_tracker['eventTrackerArn']))
        personalize.delete_event_tracker(eventTrackerArn = event_tracker['eventTrackerArn'])
        event_trackers += 1
        
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time and event_trackers > 0:
    print('Waiting for event trackers to be deleted...')
    time.sleep(10)
    event_trackers = 0
    event_trackers_paginator = personalize.get_paginator('list_event_trackers')
    for event_tracker_page in event_trackers_paginator.paginate(datasetGroupArn = dataset_group_arn):
        event_trackers += len(event_tracker_page['eventTrackers'])
        
print('Done deleting event trackers')

### Clear Event Tracker SSM Parameter

Just as with the campaign ARNs, we need to clear the SSM parameter for our event tracker ID. **However, in order for the Retail Demo Store Web UI to stop sending events to this event tracker, a rebuild of the Web UI service must be triggered in CodePipeline.**

In [ ]:
ssm.put_parameter(
    Name='retaildemostore-personalize-event-tracker-id',
    Description='Retail Demo Store Personalize Event Tracker ID Parameter',
    Value='NONE',
    Type='String',
    Overwrite=True
)

## Delete Datasets

Next, we can delete the datasets for our dataset group.

In [ ]:
print('Deleting datasets for dataset group')
dataset_arns_deleted = []
dataset_paginator = personalize.get_paginator('list_datasets')

for dataset_page in dataset_paginator.paginate(datasetGroupArn = dataset_group_arn):
    for dataset in dataset_page['datasets']:
        print('Deleting dataset {}'.format(dataset['datasetArn']))
        personalize.delete_dataset(datasetArn = dataset['datasetArn'])
        dataset_arns_deleted.append(dataset['datasetArn'])
        
if len(dataset_arns_deleted) > 0:
    print('Done initiating deletes for {} datasets(s)'.format(len(dataset_arns_deleted)))
else:
    print('No datasets to delete')

### Wait for Datasets to be Deleted

In [ ]:
print('Waiting for {} datasets to be fully deleted'.format(len(dataset_arns_deleted)))
while len(dataset_arns_deleted) > 0:
    try:
        status = None
        max_time = time.time() + 3*60*60 # 3 hours
        while time.time() < max_time:
            describe_dataset_response = personalize.describe_dataset(
                datasetArn = dataset_arns_deleted[0]
            )
            status = describe_dataset_response["dataset"]["status"]
            print('Dataset {}: {}'.format(dataset_arns_deleted[0], status))
            time.sleep(10)
            
        print('Exceeded wait time for dataset {} to delete; skipping'.format(dataset_arns_deleted[0]))
        dataset_arns_deleted.pop(0)

    except ClientError as e:
        error_code = e.response['Error']['Code']
        if error_code == 'ResourceNotFoundException':
            print('Dataset {} no longer exists'.format(dataset_arns_deleted[0]))
            dataset_arns_deleted.pop(0)
            
print('Done deleting datasets')

## Delete Dataset Group

Finally, we can delete our dataset group.

In [ ]:
print('Deleting dataset group')
personalize.delete_dataset_group(datasetGroupArn = dataset_group_arn)

### Wait for Dataset Group to be Deleted

In [ ]:
print('Waiting for dataset group to be fully deleted')
try: 
    status = None
    max_time = time.time() + 3*60*60 # 3 hours
    while time.time() < max_time:
        describe_dataset_group_response = personalize.describe_dataset_group(
            datasetGroupArn = dataset_group_arn
        )
        status = describe_dataset_group_response["datasetGroup"]["status"]
        print("Dataset group: {}".format(status))
        time.sleep(10)
        
except ClientError as e:
    error_code = e.response['Error']['Code']
    if error_code == 'ResourceNotFoundException':
        print("Dataset group fully deleted")

## Delete Schemas

We're almost done. The last step to cleaning up Personalize resources is to delete the schemas for our datasets.

In [ ]:
schemas_to_delete = [ 'retaildemostore-schema-users', 'retaildemostore-schema-items', 'retaildemostore-schema-interactions' ]
schema_paginator = personalize.get_paginator('list_schemas')
for schema_page in schema_paginator.paginate():
    for schema in schema_page['schemas']:
        if schema['name'] in schemas_to_delete:
            print('Deleting schema {}'.format(schema['schemaArn']))
            personalize.delete_schema(schemaArn = schema['schemaArn'])
            
print('Done deleting schemas')

## Delete IAM Role

Finally, we'll cleanup the IAM role we created to allow Personalize to access the S3 bucket where we placed our CSVs.

In [ ]:
!aws iam detach-role-policy --role-name RetailDemoStorePersonalizeS3Role --policy-arn arn:aws:iam::aws:policy/AmazonS3ReadOnlyAccess
!aws iam delete-role --role-name RetailDemoStorePersonalizeS3Role

## Cleanup Complete

All resources created by the Personalize workshop have been deleted.